In [1]:
import os
import subprocess
from pathlib import Path
from io import BytesIO
from concurrent.futures import ProcessPoolExecutor, as_completed

import pandas as pd

In [2]:
def predict_serotype(fasta, db='/home/chen1i6c04/Tools/LisSero/db'):
    cmd = f"lissero -s {db} {fasta}"
    child_process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)
    stdout, stderr = child_process.communicate()
    return stdout

In [3]:
dirpath = '/media/NGS/SRA_1/NCBI_Listeria_monocytogenes_SRA/Contigs'
dirPath = Path(dirpath)

In [4]:
results = list(ProcessPoolExecutor(48).map(predict_serotype, dirPath.iterdir(), chunksize=8))

In [9]:
dfs = []
for result in results:
    try:
        df = pd.read_csv(BytesIO(result), sep='\t')
        dfs.append(df)
    except:
        pass

In [12]:
serotype = pd.concat(dfs, ignore_index=True)

In [13]:
serotype.to_csv('/media/NGS/SRA_1/NCBI_Listeria_monocytogenes_SRA/serotype.csv', index=False)

In [ ]:
result = predict_serotype('/media/NGS/Data_Analysis/20191128_CDC_MiSeq/Listeria_monocytogenes/Export/*', db='/home/chen1i6c04/Tools/LisSero/db')

In [ ]:
df = pd.read_csv(BytesIO(result), sep='\t')

In [ ]:
df['ID'] = [os.path.splitext(os.path.basename(i))[0] for i in df['ID']]

In [ ]:
df.to_csv('/media/NGS/Data_Analysis/20191128_CDC_MiSeq/Listeria_monocytogenes/Serotype.csv', index=False)